# Example to use the pretrained models

In this notebook we show how to load the dataset and the pretrained models to reproduce some of the results reported in the paper, to do so we leverage the ``torch.hub`` API

In [1]:
import torch
from tqdm import tqdm
from collections import defaultdict
import pandas as pd

HUBCONF_URI = "."
HUBCONF_SOURCE = "local"

### Load the dataset

We provide implementation for DTU and Blended-MVS / Blended-MVG, once that the dataset is locally available it can be simply loaded by means of the torch.hub function

In [2]:
# load the dataset
dm = torch.hub.load(
    HUBCONF_URI,
    "blended_mvg",
    source=HUBCONF_SOURCE,
    root="data/blended-mvs",
    hints="mvguided_filtered",
    hints_density=0.03,
)
dm.prepare_data()
dm.setup()
dl = dm.test_dataloader()

### Load the Network(s)

Here we load the pretrained network trained with and without sparse depth points and test them, to reproduce the results provided in the [paper](https://arxiv.org/pdf/2210.11467v1.pdf)

In [3]:
results = defaultdict(lambda: [])

def metrics(pred: torch.Tensor, gt: torch.Tensor):
    mask = (gt > 0)
    diff = torch.abs(gt[mask] - pred[mask])
    return {
        ">1 px": torch.mean((diff > 1).float()),
        ">2 px": torch.mean((diff > 2).float()),
        ">3 px": torch.mean((diff > 3).float()),
        ">4 px": torch.mean((diff > 4).float()),
    }

for model_name in [
    "mvsnet",
    "ucsnet",
    "d2hc_rmvsnet",
    "patchmatchnet",
    "cas_mvsnet",
]:

    model_orig = torch.hub.load(
        HUBCONF_URI,
        model_name,
        source=HUBCONF_SOURCE,
        dataset="blended_mvg",
        hints="not_guided",
    )
    model_orig.eval()
    model_orig.cuda()  # use a gpu for this

    model_hints = torch.hub.load(
        HUBCONF_URI,
        model_name,
        source=HUBCONF_SOURCE,
        dataset="blended_mvg",
        hints="mvguided_filtered",
        hints_density=0.03,
    )
    model_hints.eval()
    model_hints.cuda()

    with torch.no_grad():
        for ex in tqdm(dl, desc=model_name):

            # compute inputs
            inp_no_hints = {
                "imgs": ex["imgs"]["stage_0"].cuda(),
                "intrinsics": ex["intrinsics"].cuda(),
                "extrinsics": ex["extrinsics"].cuda(),
                "depth_values": ex["depth_values"].cuda(),
            }

            inp_hints = dict(
                **inp_no_hints,
                hints=ex["hints"].cuda(),
            )
            
            # forward
            depth_orig = model_orig(**inp_no_hints)
            depth_hints = model_hints(**inp_hints)

            # metrics
            gt = ex["depth"]["stage_0"].cuda()
            results[model_name].append(metrics(depth_orig, gt))
            results[model_name + "_hints"].append(metrics(depth_hints, gt))

cas_mvsnet: 100%|██████████| 915/915 [04:30<00:00,  3.38it/s]


In [4]:
def mean_metrics(lst_d):
    return {k: f"{torch.stack([dic[k] for dic in lst_d]).mean().item():.3f}" for k in lst_d[0]}

outs = [dict(model=net_name, **mean_metrics(results[net_name])) for net_name in results]
pd.DataFrame(outs).set_index("model")

,>1 px,>2 px,>3 px,>4 px
model,,,,
mvsnet,0.145,0.076,0.048,0.033
mvsnet_hints,0.077,0.037,0.023,0.015
ucsnet,0.083,0.042,0.027,0.019
ucsnet_hints,0.040,0.018,0.011,0.008
d2hc_rmvsnet,0.190,0.102,0.065,0.044
d2hc_rmvsnet_hints,0.082,0.041,0.026,0.018
patchmatchnet,0.083,0.041,0.026,0.018
patchmatchnet_hints,0.065,0.034,0.022,0.016
cas_mvsnet,0.083,0.040,0.025,0.018
